In [1]:
%pip install equinox openmdao[all]

Note: you may need to restart the kernel to use updated packages.


In [2]:
import equinox as eqx
from util import generate_base_model
import jax.numpy as jnp

In [3]:
base_model = generate_base_model()

lift_model = eqx.tree_deserialise_leaves(f"models/lift_surrogate.eqx", base_model)
drag_model = eqx.tree_deserialise_leaves(f"models/drag_surrogate.eqx", base_model)

In [4]:
from optimizer import Optimizer

In [5]:
opt = Optimizer()

In [6]:
opt.load_surrogates(lift_model, drag_model)

In [7]:
results = opt.solve_wing(
    lift_goal=400_000,
    v_infty=300,
    rho=jnp.float32(0.5),
    mu=jnp.float32(1.46e-5),
    alpha_geo=jnp.deg2rad(jnp.float32(5))
)

Optimization terminated successfully    (Exit mode 0)
            Current function value: 11065.441748513022
            Iterations: 3
            Function evaluations: 4
            Gradient evaluations: 3
Optimization Complete
-----------------------------------
Optimization terminated successfully    (Exit mode 0)
            Current function value: 0.08020106432043735
            Iterations: 42
            Function evaluations: 65
            Gradient evaluations: 42
Optimization Complete
-----------------------------------


In [8]:
results

{'B': array([1.71150756]),
 'T': array([0.2110977]),
 'P': array([3.5]),
 'C': array([0.22667376]),
 'E': array([0.6]),
 'R': array([0.02]),
 'b': array([21.2874551]),
 'c': array([1.42001708])}